## Imports

In [453]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Class

In [454]:
class value():
    def __init__(self, data, _children = (), _op = None, label = ''):
        self.data = data
        self.grad = 0
        
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label
    
    def __add__(self,other):
        other = other if isinstance(other,value) else value(other)  
        out = value(self.data + other.data, (self,other), '+')

        def _backward():
            self.grad +=  out.grad
            other.grad += out.grad
        out._backward = _backward
        return out  

    def exp(self):
        x = self.data
        t =  np.exp(x)
        out = value(t, (self,), 'exp')
        def _backward():
            self.grad += t * out.grad
        out._backward = _backward
        return out
    
    def div(self, other):
        other = other if isinstance(other,value) else value(other)  
        out = value(self.data / other.data, (self,other), '/')
        def _backward():
            self.grad += 1 / other.data * out.grad
            other.grad += -self.data / (other.data ** 2) * out.grad
        out._backward = _backward
        return out
    

    def __mul__(self,other):
        other = other if isinstance(other,value) else value(other) 
        out = value(self.data * other.data, (self,other), '*')
        def _backward():    
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    
        
    def __pow__(self,other):    
        assert isinstance(other, (int,float)), "Power must be a scalar"
        out = value(self.data ** other, (self,), f'**{other}')
        
        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad  
        out._backward = _backward
        return out
    
    def relu(self):
        out = value(0 if self.data < 0 else self.data, (self,), 'ReLU')

        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward

        return out
    
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.grad = 1
        for v in reversed(topo):
            v._backward()

    def __neg__(self): # -self
        return self * -1

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

## Graph

In [455]:
import graphviz  
from IPython.display import display, SVG

def get_nodes_edges(root):
    # build a set upt with all the nodes and edges
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_graph(root):
    ps = graphviz.Digraph(format = 'svg', graph_attr = {'rankdir' : 'LR'} )
    
    nodes, edges = get_nodes_edges(root)

    for n in nodes:
        uid = str(id(n))
        #for any value in the graph, create a rectangle with the data of the value
        ps.node(uid, label = "{%s | data %.3f | grad %.3f}" % (n.label, n.data, n.grad), shape = 'record') 
        
        if n._op:
            #if this value is a result of an operation, create a circle with the operation
            ps.node(name =uid + n._op, label = n._op)
            #create an edge between the value and the operation
            ps.edge(uid + n._op, uid)

    for n1, n2 in edges:
        #connect the nodes
        ps.edge(str(id(n1)), str(id(n2)) + n2._op )

    svg_data = ps.pipe(format='svg')
    display(SVG(svg_data))

## Neuron and layers Class

In [456]:
class Neurons():
    def __init__(self, nin ):
        self.w = [value(np.random.uniform(-1,1)) for _ in range(nin)]
        self.b = value(0)


    def __call__(self, x):
        act = sum((wi*xi for wi,xi in zip(self.w, x)), self.b)
        return act.relu() 
    
    def parameters(self):
        return self.w + [self.b]
    
    def __repr__(self):
        return f"{'ReLU' if self.nonlin else 'Linear'}Neuron({len(self.w)})"
    
class Layer():
    def __init__(self, nin, nout):
        self.neurons = [Neurons(nin) for _ in range(nout)] 

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]    
    
    def __repr__(self):
        return f"Layer of [{', '.join(str(n) for n in self.neurons)}]"

class MLP():
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for l in self.layers for p in l.parameters()]


In [457]:
x = [2.0, 3.0, -2.0]
n = MLP(3, [4, 4, 1])

In [458]:
xs = [
    [2.0, 3.0, -2.0],
    [1.0, 2.0, 3.0],
    [2.0, 3.0, -2.0],
    [1.0, 2.0, 3.0]
    ]
ygt = [1.0, -1.0, -1.0, 1.0]

In [459]:
y_pred = [n(x) for x in xs]
loss = sum((ygt - yout)**2 for ygt, yout in list(zip(ygt, y_pred)))
loss

Value(data=14.48921173345732, grad=0)

In [460]:
y_pred

[Value(data=0.6574219854790766, grad=0),
 Value(data=2.193718805986175, grad=0),
 Value(data=0.6574219854790766, grad=0),
 Value(data=2.193718805986175, grad=0)]

In [461]:
loss.backward()

In [462]:
# update parameters
for p in n.parameters():
    p.data -= 0.01 * p.grad


In [463]:
for k in range(20):

    #forward pass
    y_pred = [n(x) for x in xs]
    loss = sum((ygt - yout)**2 for ygt, yout in list(zip(ygt, y_pred)))

    #backward pass
    for p in n.parameters():
        p.grad = 0
    loss.backward()

    #update parameters
    for p in n.parameters():
        p.data += -0.001 * p.grad

    print(k, loss.data)

0 4.0
1 4.0
2 4.0
3 4.0
4 4.0
5 4.0
6 4.0
7 4.0
8 4.0
9 4.0
10 4.0
11 4.0
12 4.0
13 4.0
14 4.0
15 4.0
16 4.0
17 4.0
18 4.0
19 4.0
